In [1]:
#数据透视表可以看做在二维表格上的累计操作（行列同时分组）
import numpy as np
import pandas as pd

titanic = pd.read_csv('titanic.csv')
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [2]:
#手工制作数据透视表
#1、统计不同性别乘客的生还率
titanic.groupby('sex')['survived'].mean()
#titanic.groupby('sex')[['survived']].mean()

sex
female    0.742038
male      0.188908
Name: survived, dtype: float64

In [3]:
titanic.groupby('sex')[['survived']].mean()
#对比一下区别，如果只用一个[]，这样输出的是Series对象,两个[]，输出dataframe对象,行索引转列索引

,survived
sex,
female,0.742038
male,0.188908


In [5]:
#2、查看不同船舱等级，不同性别乘客的生还率
titanic.groupby(['sex','class'])['survived'].aggregate('mean').unstack()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [6]:
#在dataframe中有专门的数据透视表语法,累计函数默认为mean
titanic.pivot_table('survived',index='sex',columns='class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


In [7]:
#当然，也可以设置多级数据透视表，比如，对年龄进行分段
age = pd.cut(titanic['age'],[0,18,80])
titanic.pivot_table('survived',['sex',age],'class')

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 80]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 80]  0.375000  0.071429  0.133663

In [8]:
#同样，也可以用qcut将船票价格这一属性等分成两半,做一个四维数据透视表
fare = pd.qcut(titanic['fare'],2)
titanic.pivot_table('survived',['sex',age],[fare,'class'])

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (0, 18]               NaN  1.000000  0.714286          0.909091   
       (18, 80]              NaN  0.880000  0.444444          0.972973   
male   (0, 18]               NaN  0.000000  0.260870          0.800000   
       (18, 80]              0.0  0.098039  0.125000          0.391304   

fare                                 
class              Second     Third  
sex    age                           
female (0, 18]   1.000000  0.318182  
       (18, 80]  0.914286  0.391304  
male   (0, 18]   0.818182  0.178571  
       (18, 80]  0.030303  0.192308

In [9]:
#数据透视表的完整语法签名为
# dataframe.pivot_table(data,values=None,index=None,columns=None,aggfunc='mean',fill_value=None,margins=False,
#                      dropna=True,margin_name='All')
#aggfunc用于指明聚合的具体函数，可以用字典指定不同列不同指定函数，此时会省略参数values,因为aggfunc已经确定了数值
titanic.pivot_table(index='sex',columns='class',
                   aggfunc={'survived':sum,'fare':'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

In [10]:
#margins参数用于返回每一组的汇总情况，即分组表的汇总列，默认名称为‘ALL’
titanic.pivot_table('survived',index='sex',columns='class',margins=True)

class,First,Second,Third,All
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
All,0.629630,0.472826,0.242363,0.383838
